In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!apt-get update
!apt-get install -y git

In [ ]:
!git clone https://github.com/meronheron/Chain-of-Tools.git

In [ ]:
!ls /kaggle/working

In [ ]:
# !cp -r /kaggle/input/chain-of-tools-repo /kaggle/working/Chain-of-Tools

In [ ]:
!ls /kaggle/working/Chain-of-Tools

In [ ]:
# !ls /kaggle/working/Chain-of-Tools/Chain-of-Tools

In [ ]:
# !rm -rf /kaggle/working/Chain-of-Tools

In [ ]:
# !cp -r /kaggle/input/chain-of-tools-repo/Chain-of-Tools/* /kaggle/working/Chain-of-Tools/

In [ ]:
# !mkdir -p /kaggle/working/Chain-of-Tools

In [ ]:
# !cp -r /kaggle/input/chain-of-tools-repo/Chain-of-Tools/* /kaggle/working/Chain-of-Tools/

In [ ]:
!ls /kaggle/working/Chain-of-Tools

In [ ]:
!sed -i '/torch==2.2.1+cu118/d; /torchaudio==2.2.1+cu118/d; /torchvision==0.17.1+cu118/d' /kaggle/working/Chain-of-Tools/requirements.txt

In [ ]:
!pip install -r /kaggle/working/Chain-of-Tools/requirements.txt

In [ ]:
!pip install torchvision==0.17.1

In [ ]:
!pip list | grep -E 'accelerate|beautifulsoup4|einops|huggingface-hub|numpy|safetensors|tokenizers|torchvision'

In [ ]:
!ls /kaggle/working/Chain-of-Tools

In [ ]:
!ls /kaggle/working/Chain-of-Tools/src

In [ ]:
!python /kaggle/working/Chain-of-Tools/src/main.py

In [ ]:
!ls /kaggle/working/Chain-of-Tools/config

In [ ]:
!mkdir -p /kaggle/working/config
!cp /kaggle/working/Chain-of-Tools/config/basic.yaml /kaggle/working/config/

In [ ]:
!mkdir -p /kaggle/working/my_log

In [ ]:
!sed -i 's|checkpoint: ../PTM/Llama-2-7b-chat-hf|checkpoint: meta-llama/Llama-2-7b-chat-hf|' /kaggle/working/config/basic.yaml

In [ ]:
!pip install huggingface_hub


In [ ]:
from kaggle_secrets import UserSecretsClient; 
from huggingface_hub import login; 

login(token=UserSecretsClient().get_secret('HF_TOKEN'))

In [ ]:
!cat /kaggle/working/config/basic.yaml | grep checkpoint

In [ ]:
python ./src/script/train_gsm8k_retriever.py --config_file config/train_gsm8k_retriever.yaml


In [ ]:
python ./src/script/train_gsm8k_judge.py --config_file config/train_judge.yaml


In [ ]:
python ./src/script/test_gsm8k.py --config_file config/test.yaml


In [ ]:
!python /kaggle/working/Chain-of-Tools/src/main.py